In [1]:
# Required libraries
import simpy
import numpy as np
import pandas as pd
import random
from modsim import *
import statistics

In [2]:
wait_times = []

In [3]:
class bike_shop(object):
    def __init__(self, env, num_repairs, num_sales, num_random_inq):
        self.env = env
        #self.floor_person = simpy.Resource(env, num_floor_person)
        self.repairs = simpy.Resource(env, num_repairs)
        self.sales = simpy.Resource(env, num_sales)
        self.random_inq = simpy.Resource(env, num_random_inq)
        #self.storage = simpy.Resource(env, num_storage)
        #self.exit = exit
        
    # Setting time it takes for a customer to meet with floor person   
    #def meet_floor_person(self, customer):
        # min of 2 minutes and max of 10 minutes
        #yield self.env.timeout(random.randint(2, 11)) 
        
    # Setting time it takes for customer to be in repairs
    def repairs(self, customer):
        # min of 10 minutes and max of 3 days in minutes
        yield self.env.timeout(random.randint(10, 4321)) 
    
    # Setting time it takes for customer to be in sales
    def sales(self, customer):
        # min of 15 minutes and max of 45 minutes
        yield self.env.timeout(random.randint(15, 46)) 
    
    # Setting time it takes for customer to be in random_inq
    def random_inq(self, customer):
        # min of 2 minutes and max of 10 minutes
        yield self.env.timeout(random.randint(2, 11))    
    
    # Setting time it takes for customer's bike to be in storage
    #def bike_storage(self, customer):
        # min of 7 days and max of 365 days in minutes
        #yield self.env.timeout(random.randint(10080, 525601))  
        
    # Setting time it takes for customer to exit
    #def exit(self, customer):
        # min of 10 minutes and max of 1 hour in minutes
        #yield self.env.timeout(random.randint(10, 61))
        

In [4]:
# Defining customer's arrival time
def customer_arrival(env, customer, bike_shop):
    # Customer arrives to bike shop
    arrival_time = env.now

    # Starting process when customer meets with floor person
    '''
    Here we are working with a for loop to assess where customer needs
    to go: repairs, sales or if it's a random inquiry.
    '''
    if random.choice([True, False]):
        with bike_shop.repairs.request() as request:
            yield request
            yield env.process(bike_shop.repairs(customer))
    elif random.choice([True, False]):
        with bike_shop.sales.request() as request:
            yield request
            yield env.process(bike_shop.sales(customer))
    else:
        #with bike_shop.random_inq.request() as request:
        yield request
        yield env.process(bike_shop.random_inq(customer))
        
    '''
    with bike_shop.floor_person.request() as request:
        yield request
        yield env.process(bike_shop.meet_floor_person(customer))
            
    with bike_shop.storage.request() as request:
        yield request
        yield env.process(bike_shop.bike_storage(customer))

    with bike_shop.exit.request() as request:
        yield request
        yield env.process(bike_shop.exit(customer))
    '''    
            
    # Customer heads to choice
    wait_times.append(env.now - arrival_time)


In [5]:
def gets_service(env, num_repairs, num_sales, num_random_inq):
    service = bike_shop(env, num_repairs, num_sales, num_random_inq)
    
    for customer in range(3):
        env.process(customer_arrival(env, customer, bike_shop))
        
                 
    # Wait a bit before generating another customer
    while True:
        yield env.timeout(10) # 5 minutes
        
        # Incrementing customer by 1 and generating the next customer
        customer += 1
        env.process(customer_arrival(env, customer, bike_shop))

In [6]:
# Calculating wait time

def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)
     # Print the results
    minutes, frac_minutes = divmod(average_wait, 10)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)

In [7]:
# Choosing parameters

def get_user_input():
    #num_floor_person = input("Input # of people working the floor: ")
    num_repairs = input("Input # of repairs being worked on: ")
    num_sales= input("Input # of sales done: ")
    num_random_inq = input("Input # of random inquiries: ")
    #num_storage = input("Input # of bikes in storage: ")
    #num_exit = input("Input # of customers that exit the shop: ")
    params = [num_repairs, num_sales, num_random_inq]
    if all(str(i).isdigit() for i in params):  # Check input is valid
        params = [int(x) for x in params]
    else:
        print(
            "Could not parse input. The simulation will use default values:",
            "\n1 repairs, 1 sales, 1 random inquiries.",
        )
        params = [1, 1, 1]
    return params


In [8]:
# Finalizing the setup
def main():
    # Setup
    random.seed(42)
    num_repairs, num_sales, num_random_inq = get_user_input()
    
    # Run the simulation
    env = simpy.Environment()
    env.process(gets_service(env, num_repairs, num_sales, num_random_inq))
    env.run(until = 480) # running the process for 8 hours in minutes
    
    # View the results
    mins, secs = get_avarage_wait_time(wait_times)
    print("Running simulation...", f"\nThe average wait time is {mins} minutes and {secs} seconds.",)


In [9]:
# Run the simulation
if __name__ == "__main__":
    main()

Input # of repairs being worked on: 1
Input # of sales done: 1
Input # of random inquiries: 1


AttributeError: 'function' object has no attribute 'request'